---
# Recommender Systems
[Yukun Liu](mailto:yukun.liu@telecom-paris.fr), [Xiuhan Su](mailto:xiuhan.su@telecom-paris.fr)
___

> ## 1. Presentation of the model
> #### Question 1.1 
>Download the file and check size of R

In [104]:
from movielens_utils import *
from scipy.optimize import check_grad
from scipy.sparse.linalg import svds
import os
import pandas as pd

In [2]:
# filename = "D:/Telecom 1e anne/Telecom Period 2/SD-TSIA211/TP/ml-100k/u.data"
filename = os.getcwd() + '\\ml-100k\\u.data'
[R, mask] = load_movielens(filename, minidata=False)
[mini_R, mini_mask] = load_movielens(filename, minidata=True)

print('The size of R is {}'.format(R.shape))
print('The mini size of R is {}'.format(mini_R.shape))

The size of R is (943, 1682)
The mini size of R is (100, 200)


R has an original size $(943,1682)$, and it reduces to $(100,200)$ when option **minidata** is on. **minidata** is used to reduce the matrix size R, which can process the iteration speed.

> #### Question 1.2
>How many user and flms are there in the database ? What is the total number of grades ?

In [3]:
np.sum(mask)

100000

There are 943 users and 1682 films in the database, and the total number of grades is 10000.

> ## 2. Find $P$ when $Q^0$ is fixed
> #### Question 2.1
>Calculate the gradient of function $g$. We will admit that this gradient is Lipschitz continuous with constant $L_0 = \rho+||Q^0)^TQ^0||_F$ .

According to the context, we know
$$g(P)=\frac{1}{2}\mathop{\Sigma}_{u\in U, i \in I}(R_{u,i}-(\mathbb{1}_K)_{u,i}\mathop{\Sigma}_{f\in F}Q_{u,f}P_{f,i})^2+\frac{\rho}{2}\mathop{\Sigma}_{u\in U, f \in F}Q_{u,f}^2+\frac{\rho}{2}\mathop{\Sigma}_{i\in I, f \in F}P_{f,i}^2$$

Furthermore its gradient $\triangledown g=\frac{\partial g(P)}{\partial P}$ is a $F$ by $I$ matrix. While the (m,n) element of gradient $\triangledown g$ can be denoted by

$$\begin{aligned}
[\triangledown g][\triangledown g]_{m,n}&=\frac{\partial g(P)}{\partial P_{m,n}}\\
&=\mathop{\Sigma}_{u\in U}Q^0_{u,m}(\mathbb{1}_K)_{u,n}((\mathbb{1}_K)_{u,n}\mathop{\Sigma}_{f\in F}Q^0_{u,f}P_{f,n}-R_{u,n})+\rho P_{m,n}\\
% &=\mathop{\Sigma}_{u\in U}(Q^0)^T_{m,u}(\mathbb{1}_K)_{u,n}(\mathop{\Sigma}_{f\in F}Q^0_{u,f}P_{f,n}-R_{u,n})+\rho P_{m,n}\\
&=\mathop{\Sigma}_{u\in U}(Q^0)^T_{m,u}(\mathbb{1}_K)_{u,n}(Q^0P)_{u,n}-(Q^0)^T_{m,:}R_{:,n}+\rho P_{m,n}\\
&=\mathop{\Sigma}_{u\in U}(Q^0)^T_{m,u}(\mathbb{1}_K\circ Q^0P)_{u,n}-(Q^0)^T_{m,:}R_{:,n}+\rho P_{m,n}\\
% &=\mathop{\Sigma}_{f\in F}\mathop{\Sigma}_{u\in U}(Q^0)^T_{m,u}Q^0_{u,f}(\mathbb{1}_K)_{u,n}P_{f,n}-(Q^0_{m,:})^TR_{:,n}+\rho P_{m,n}\\
% &=\mathop{\Sigma}_{f\in F}[(Q^0)^TQ^0]_{m,f}P_{f,n}(\mathbb{1}_K)_{u,n}-(Q^0_{m,:})^TR_{:,n}+\rho P_{m,n}\\
% &=[(Q^0)^T]_{m,:}(\mathbb{1}_K\circ Q^0P)_{:,n}-(Q^0)^T_{m,:}R_{:,n}+\rho P_{m,n}\\
&= [(Q^0)^T(\mathbb{1}_K\circ Q^0P)]_{m,n}-[(Q^0)^TR]_{m,n}+\rho P_{m,n}
\end{aligned}$$

And the gradient can be formulated as
$$\begin{aligned}
\triangledown g(P) &= (Q^0)^T(\mathbb{1}_K\circ Q^0P)-(Q^0)^TR+\rho P\\
&= (Q^0)^T(\mathbb{1}_K\circ Q^0P-R)+\rho P
\end{aligned}$$

> #### Question 2.2
>The function objective provided in the file _movielens utils.py_ computes g(P). Complete
this function so that it also computes $\triangledown g(P)$. You may check your calculations with the
function _scipy.optimize.check grad_ (you may need _numpy.reshape_ and _numpy.ravel_
because check grad does not accept matrix variables).

In [3]:
def func(P):
    '''
    cost function w.r.t Q
    P : column vector of size(1, F*C)
    mask, R taken as given
    '''
    if P.size == 6728:
        P = P.reshape(4, 1682)
        
    tmp = (R - Q0.dot(P)) * mask
    value = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    return value

def grad(P, ravel=True):
    '''
    gradient function w.r.t Q
    P : column vector of size(1, F*C)
    ravel : True when P is ravelled to a column vector
    '''
    
    if ravel:
        P = P.reshape(4, 1682)
        tmp = (R - Q0.dot(P)) * mask 
        grad_P = np.ravel(Q0.T.dot(-tmp) + rho * P)

    else:
        tmp = (R - Q0.dot(P)) * mask 
        grad_P = Q0.T.dot(-tmp) + rho * P            

    return  grad_P   

In [4]:
rho = 0.5

Q0,s,P0 = svds(R, k=4)

[val, grad_P] = objective(P0, Q0, R, mask, rho)

err = check_grad(func, grad, np.ravel(P0))

print('The gradient error is {}'.format(err))
print('The norm is {}'.format( np.linalg.norm(grad_P) ))

KeyboardInterrupt: 

Considering the norm of gradient, the error is relatively small. Then a conclusion can be drawn that our function of gradient is derived in the right way.

> #### Question 2.3 
>Code a function `gradient(g, P0, gamma, epsilon)` that minimizes a function g using the gradient method with a constant step size, $\gamma$, starting from the initial point $P^0$ and with stopping criterion $||\triangledown g(P_k)||_F\le \epsilon$.

>#### Question 2.4
Run the function coded in the previous question in order to minimize g up to the precision $\epsilon = 1$.

Since the gradient is Lipschitz continuous with constant $L_0 = \rho+||Q^0)^TQ^0||_F$, the constant step size will be set as $$\gamma = \frac{2}{L_0}$$

The sequence of points $(P_k)_{k \in \mathbb{N}}$ in $\mathbb{R}^{F\times I}$ will be defined by
$$P_{k+1}=P_k-\gamma \triangledown g(P_k)$$

In [5]:
def gradient(P, gamma, epsilon, a = None, bcoef = None):
    """
    minimizes a function g using the gradient method
    P : initial point
    gamma : constant step size
    epsilon : exit criterion
    lsch : True when applying Taylor based line search
    """
    
    counter = 0 # step size
    g = grad(P, ravel=False)   
    
    while np.linalg.norm(g) > epsilon :
        counter += 1            

        if a != None and bcoef != None:
            """
            line search method calculating step size
            """
            gamma = find_step(g, P, gamma, a, bcoef)  
              
        P = P - gamma * g            
        g = grad(P, ravel=False)

    return g, counter

def find_step(g, P, gamma0, a, bcoef):
    b = bcoef * gamma0
    for l in range(0,100):
        
        gamma = b * a**l
        measure = func(P) - func(P - gamma*g) - 1/2 * gamma * np.linalg.norm(g)**2

        if measure > 0:
            break

    return gamma

In [6]:
# Parameter setting
L0 = rho + np.linalg.norm( Q0.T.dot(Q0) )
Q0,s,P0 = svds(R, k=4)
Gamma  = 2 / L0
G0 = grad(P0, ravel=False)
Epsilon = 1

In [7]:
# constant step 
gout = gradient(P0, Gamma, Epsilon)

print('The precision is {}'.format(np.linalg.norm(gout[0])))
print('The optimality is achieved in {} steps'.format(gout[1]))

The precision is 0.6258549235496634
The optimality is achieved in 8 steps


> #### Question 2.5
Add a line search to your gradient method, so that you do not rely on the Lipschitz constant of the gradient any more.

Applying the Talyor-based line search, we have

$$g(P^+(\gamma_k))\le g(P_k)+\langle\triangledown g(P_k), P^+(\gamma_k)-P_k\rangle+\frac{1}{2\gamma_k}\|P_k-P^+(\gamma_k)\|^2,k\in\{1,\cdots,N\}$$

where $P^+(\gamma_k)=P_k-\gamma_k\triangledown g(P_k)$ and $\gamma_k = ba^l,b>0, a\in(0,1)$, $l$ is bounded by the criterion that $L'=\frac{1}{\gamma_k}\ge L$.

Since $P^+(\gamma_k)-P_k=-\gamma_k\triangledown g(P_k)$, the inequality can be formulated as 

$$ g(P_k)-g(P^+(\gamma_k))-\frac{\gamma_k}{2}\|\triangledown g(P_k)\|^2\ge 0$$ $l$ is the smallest integer makes the inequality true. In the implementation, we chose the classical parameter: $a=0.5,b=2\gamma_k$

<!-- Let's set $$\gamma_k = ba^l,k\in\{1,\cdots,N\}$$ where $a=0.5, b=2\gamma_{k-1},l=\lfloor \frac{\log(ba^{-1}L_0)}{\log(a^{-1})}\rfloor, \gamma_0=\frac{2}{L_0}$ to ensure the chosing step won't break the Lipschitz continuity.

Since $L_0 = \rho+||Q^0)^TQ^0||_F=2.5, \gamma_0=\frac{2}{L_0}=0.8,$ $l$ is chosen to be $1$ in this case. Besides, for those cases $L_0$ is unknown, $l=1$ will always be the least choice which indicates constant step size.
Then 
$$P^+(\gamma_k)=P_k-\gamma_k\triangledown g(P_k)$$ -->

In [8]:
# linsearch
gout_lsch = gradient(P0, Gamma, Epsilon, a=0.5, bcoef=2)

print('The precision is {}'.format(np.linalg.norm(gout_lsch[0])))
print('The optimality is achieved in {} steps'.format(gout_lsch[1]))

The precision is 0.25486949872124526
The optimality is achieved in 7 steps


> ## Resolution of the full problem
> #### Question 3.1 
Let f be the function defined by $f(P,Q) = \frac{1}{2}||\mathbb{1}_K \circ(R-QP)||^2_F+\frac{\rho}{2}||Q||^2_F+\frac{\rho}{2}||P||^2_F$. 
>
>By remarking that f is a polynomial of degree 4, show that its gradient is not Lipschitz
continuous.

The function $f$ is expressed as
$$f(P,Q)=\frac{1}{2}\mathop{\Sigma}_{u\in U, i \in I}(R^2_{u,i}-2(\mathbb{1}_K)_{u,i}R_{u,i}\mathop{\Sigma}_{f\in F}Q_{u,f}P_{f,i}+\mathop{\Sigma}_{a,b\in F}Q_{u,a}P_{a,i}Q_{u,b}P_{b,i})+\frac{\rho}{2}\mathop{\Sigma}_{u\in U, f \in F}Q_{u,f}^2+\frac{\rho}{2}\mathop{\Sigma}_{i\in I, f \in F}P_{f,i}^2$$

And we can tell the mysterious term $\mathop{\Sigma}_{a,b\in F}Q_{u,a}P_{a,i}Q_{u,b}P_{b,i}$ is with order of 4. As a result, $f$ is not Lipschitz continuous.

>#### Question 3.2
Solve Problem (1) by the gradient method with line search until reaching the precision
$\|\triangledown f(P_k,Q_k)\|_F \le\epsilon$ with $\epsilon = 100$. How do you interpret what the algorithm returns ?

The gradient of function $f$ w.r.t $Q$ can be derived in a similar way as Prob.(2.1), then we have

$$\frac{\partial f(P,Q)}{\partial P}= Q^T(\mathbb{1}_K\circ QP-R)+\rho P$$
$$\frac{\partial f(P,Q)}{\partial Q}= (\mathbb{1}_K\circ QP-R)P^T+\rho Q$$
while $\frac{\partial f(P,Q)}{\partial P}\in\mathbb{R}^{F\times I},\frac{\partial f(P,Q)}{\partial Q}\in\mathbb{R}^{U\times F}$.
And the overall gradient is 
$$\triangledown f(P,Q)=\
\begin{bmatrix}\frac{\partial f(P,Q)}{\partial P}\\ \frac{\partial f^T(P,Q)}{\partial Q}
\end{bmatrix}$$

Similary, to implement the Taylor based line search, we'll find the step $\gamma_k=ba^l$ s.t.
$$ f(P_k,Q_k)-f(P^+(\gamma_k),Q^+(\gamma_k))-\frac{\gamma_k}{2}\|\triangledown f(P_k,Q_k)\|^2\ge 0$$ $l$ is the smallest integer makes the inequality true.

In [9]:
val0, grad_P0, grad_Q0 = total_objective(P0, Q0, R, mask, rho)

PQvec_ini = np.concatenate([grad_P0.ravel(), grad_Q0.ravel()])
val0, PQvec_1 = total_objective_vectorized(PQvec_ini, R, mask, rho)
val1, PQvec_2 = total_objective_vectorized(PQvec_1, R, mask, rho)

In [16]:
epsilon = 100
PQvec_gradini = np.concatenate([grad_P0.ravel(), grad_Q0.ravel()])
gamma_test = 0.5
find_gamma(PQvec_gradini, gamma_test, 0.5, 2)

-2.1843600334971337e+32
-1.3652047008696042e+31
-8.532275383971703e+29
-5.33235463031049e+28
-3.332324815372113e+27
-2.0822070420597326e+26
-1.30075961225775e+25
-8.122004473246104e+23
-5.0665845622077305e+22
-3.1545566578665094e+21
-1.9565909622169613e+20
-1.204276319132169e+19
-7.298435893671085e+17
-4.286365332688026e+16
-2360060243133986.0
-113651749583715.98
-4252755822412.793
-193661790249.08493
-43589778067.41798
-1515440198.872284
8977751350.48817
20


9.5367431640625e-07

In [27]:
def find_gamma(PQvec, gamma0, a, bcoef):
    
    b = bcoef * gamma0
    
    for l in range(0,100):
        
        gamma = b * a**l
        val0, PQvec_grad = total_objective_vectorized(PQvec, R, mask, rho)
        PQvec_new = PQvec - gamma * PQvec_grad
        
        val1, PQvec_grad1 = total_objective_vectorized(PQvec_new, R, mask, rho)
        
        measure = val0 - val1 - 1/2 * gamma * np.linalg.norm(PQvec_grad)**2

        if measure > 0:
#             print(l)
            break

    return gamma, PQvec_new, PQvec_grad

def gradient_all(P, Q = None, gamma0, epsilon, a = None, bcoef = None):
    """
    minimizes a function g using the gradient method
    P : initial point
    gamma : constant step size
    epsilon : exit criterion
    lsch : True when applying Taylor based line search
    """
    
    counter = 0 # step size
    
    PQvec = np.concatenate([P.ravel(), Q.ravel()])
    
    val0, grad_P0, grad_Q0 = total_objective(P, Q, R, mask, rho)
    
    PQvec_grad = np.concatenate([grad_P0.ravel(), grad_Q0.ravel()])
    
    while np.linalg.norm(PQvec_grad) > epsilon :
        counter += 1            

        if a != None and bcoef != None:
            """
            line search method calculating step size
            """e
            gamma0, PQvec, PQvec_grad = find_gamma(PQvec, gamma0, a, bcoef)  
              
    return np.linalg.norm(PQvec_grad), counter, PQvec

In [110]:
epsilon = 100
err, steps, PQvec = gradient_all(P0, Q0, gamma_test, epsilon, a = 0.5, bcoef = 2)

In [41]:
n_items = R.shape[1]
n_users = R.shape[0]
F = PQvec.shape[0] // (n_items + n_users)

Pvec = PQvec[0:n_items*F]
Qvec = PQvec[n_items*F:]

P = np.reshape(Pvec, (F, n_items))
Q = np.reshape(Qvec, (n_users, F))

In [94]:
re = Q.dot(P)

df = pd.DataFrame(re)

In [95]:
index =  df[300][df.iloc[300] > 4].index.tolist()

The returned $Q$ and $P$ indicate the matrixes make the term $QP$ **most close** to our observation $R$.

>#### Question 3.3
What film would you recommend to user 300 ?

In [112]:
steps, err

(131, 97.13320622890856)

In [108]:
user = 299
thres = 4.5 # the rating no less than thres will be recommended

rec_index =  df[user][df.iloc[user] > thres].index.tolist()
print(len(rec_index))

21


For User 300, we will recommend the film in index
$[99, 256, 287, 293, 299, 327]$